In [17]:
import pickle
import os
from load_data import LoadData
import time
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.metrics.pairwise import pairwise_distances
from tqdm import tqdm

In [2]:
PATH = 'C:/Users/jtogt2/Notebook/data/embeddings/'
dimensions = 6
n_neighbors = 30

In [3]:
embedding = pickle.load((open(f'{PATH}embedding_{dimensions}_{n_neighbors}', 'rb')))

In [4]:
loader = LoadData()
PATH = 'C:/Users/jtogt2/Notebook/data/'

if os.path.isfile(PATH + 'processed_dataframe.csv'):
    print("Loading saved data...")
    start = time.time()
    loader.load_processed_frame()
    dataset = loader.retrieve_measure_frame()
    print(f"Data loaded in: {time.time() - start} seconds")
else:
    print("Preprocessing data...")
    start = time.time()
    loader.load_celldyn()
    loader.preprocess()
    dataset = loader.retrieve_measure_frame()
    print(f"Data processed in: {time.time() - start} seconds")

Loading saved data...
Data loaded in: 30.934802293777466 seconds


In [5]:
subset_wi_ga = dataset
subset_wo_ga = subset_wi_ga.loc[:, [i for i in subset_wi_ga.columns if i[:3] == 'c_b']]

In [6]:
embedded_dataset = embedding.transform(subset_wo_ga)

In [7]:
df = pd.DataFrame(data=embedded_dataset,
                          index=np.arange(0,len(embedded_dataset)),
                          columns=['f' + str(i) for i in range(len(embedded_dataset[0]))])


In [8]:
PATH = 'C:/Users/jtogt2/Notebook/data/embeddings/'
labels = np.genfromtxt(f'{PATH}embedding_{dimensions}_labels.csv', delimiter=',')

In [9]:
df['cluster_assignment'] = labels
df_1 = df[df['cluster_assignment'] >= 0]

In [10]:
grouped_df = df_1.groupby('cluster_assignment')

In [12]:
li = []
for index, i in enumerate(grouped_df):
    a = i[1]
    a = a.drop('cluster_assignment', axis=1)
    li.append(a)

In [13]:
from scipy.spatial.distance import directed_hausdorff

In [32]:
ind = 0
sim_li = [[] for i in range(len(li))]
for ind, _ in tqdm(enumerate(li)):
    for index, i in enumerate(li):
        if ind != index:
            sim_li[ind].append(directed_hausdorff(i, li[ind])[0])
        else:
            sim_li[ind].append(np.inf) 

0it [00:00, ?it/s]<ipython-input-32-f19f066c6277>:6: UserWarning: `x` isn't a recognized object; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  sim_li[ind].append(directed_hausdorff(i, li[ind])[0])
77it [1:20:38, 62.84s/it] 


In [33]:
sim_li = np.array(sim_li)

In [73]:
close_li = []
for index, i in enumerate(sim_li):
    if np.min(i) < 1:
        close_li.append((index, np.argmin(i)))
close_li = np.array(close_li)

In [74]:
close_li = [sorted(i) for i in close_li]

In [75]:
close_li

[[4, 5],
 [7, 8],
 [12, 13],
 [29, 30],
 [29, 30],
 [33, 52],
 [33, 52],
 [55, 65],
 [56, 57],
 [56, 57],
 [18, 59],
 [60, 61],
 [60, 61],
 [64, 65],
 [64, 65],
 [66, 67],
 [66, 67],
 [71, 72],
 [73, 74],
 [73, 74],
 [75, 76],
 [75, 76]]

In [76]:
dupes = [x for n, x in enumerate(close_li) if x in close_li[:n]]

In [77]:
dupes

[[29, 30],
 [33, 52],
 [56, 57],
 [60, 61],
 [64, 65],
 [66, 67],
 [73, 74],
 [75, 76]]

In [78]:
from main import *

In [79]:
result = run_dimension(6, frames=True)

Loading saved data...
Data loaded in: 33.74074697494507 seconds
Loading saved embedding...
Embedding loaded in: 3.6067209243774414 seconds
HDBSCAN does not classify 19371 samples


In [80]:
sim_df = pd.DataFrame()

In [81]:
for i in dupes:
    sim_df = sim_df.append(result.loc[("mean", i[0])])
    sim_df = sim_df.append(result.loc[("mean", i[1])])
    sim_df = sim_df.append(pd.Series(0, index=sim_df.columns), ignore_index=True)

In [82]:
PATH = 'C:/Users/jtogt2/Notebook/cluster_analysis/frames/'
sim_df.to_excel(f'{PATH}sim_df.xlsx')

In [86]:
x = result.loc['mean']

In [87]:
x.to_excel(f'{PATH}result_cluster_{dimensions}_means.xlsx')